In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.patches as mpatches
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import seaborn as sns  
sns.set(color_codes = True)
sns.set(style="whitegrid")
import plotly.figure_factory as ff
from plotly.colors import n_colors

In [ ]:
df = pd.read_csv("../input/covid-world-vaccination-progress/country_vaccinations.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
(round(df.isnull().sum()/len(df)*100,2)).apply(lambda x :str(x)+" %")

In [ ]:
df.loc[df['iso_code'].isnull(),'country'].value_counts()

In [ ]:
df['iso_code'] = df['iso_code'].fillna('GBR')

Columns to drop
As we can see, 'daily_vaccinations_raw' has a cleaned variable 'daily_vaccinations' with less missing values, so we can now drop 'daily_vaccinations_raw'

In [ ]:
data = df

In [ ]:
df = df.drop('daily_vaccinations_raw', axis=1)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date', ascending=True)

df['date'] = df['date'].dt.strftime('%m-%d-%Y')

In [ ]:
df.head()

In [ ]:
unique_dates = df['date'].unique()

In [ ]:
for iso_code in df['iso_code'].unique():
    for inc_date in unique_dates:
        if df.loc[df['iso_code'] == iso_code, 'date'].str.contains(inc_date).any():
            continue
        else:
            df.loc[len(df)] = [None, iso_code, inc_date] + 11 * [None]

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date', ascending=True)

df['date'] = df['date'].dt.strftime('%m-%d-%Y')

In [ ]:
df.columns

In [ ]:
tdf = df.copy()


for iso_code in tdf['iso_code'].unique():
    tdf.loc[tdf['iso_code'] == iso_code, :] = tdf.loc[tdf['iso_code'] == iso_code, :].fillna(method='ffill').fillna(0)

In [ ]:
fig = px.choropleth(
    tdf,                            # Input Dataframe
    locations="iso_code",           # identify country code column
    color="total_vaccinations",                     # identify representing column
    hover_name="country",              # identify hover name
    animation_frame="date",
    #category_orders={"frame": list(sorted(tdf['date'].unique()))},            # identify date column
    color_continuous_scale= 'viridis',
    projection="natural earth",        # select projection
    range_color=[0,5000000],
    title='Number of vaccinations by country',
)             # select range of dataset     
fig.show() 

In [ ]:
fig = px.choropleth(
    tdf,                            # Input Dataframe
    locations="iso_code",           # identify country code column
    color="daily_vaccinations",                     # identify representing column
    hover_name="country",              # identify hover name
    animation_frame="date",
    #category_orders={"frame": list(sorted(tdf['date'].unique()))},            # identify date column
    color_continuous_scale= 'viridis',
    projection="natural earth",        # select projection
    range_color=[0,1000000],
    title='Number of daily vaccinations',
)             # select range of dataset     
fig.show() 

In [ ]:
data.columns

In [ ]:
df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')

df.fillna(value = 0, inplace = True)
df.total_vaccinations = df.total_vaccinations.astype(int)
df.people_vaccinated = df.people_vaccinated.astype(int)
df.people_fully_vaccinated = df.people_fully_vaccinated.astype(int)

df.daily_vaccinations_raw = df.daily_vaccinations_raw.astype(int)
df.daily_vaccinations = df.daily_vaccinations.astype(int)
df.total_vaccinations_per_hundred = df.total_vaccinations_per_hundred.astype(int)

df.people_fully_vaccinated_per_hundred = df.people_fully_vaccinated_per_hundred.astype(int)
df.daily_vaccinations_per_million = df.daily_vaccinations_per_million.astype(int)
df.people_vaccinated_per_hundred = df.people_vaccinated_per_hundred.astype(int)

date = df.date.str.split('-', expand =True)
date

In [ ]:
df['year'] = date[0]
df['month'] = date[1]
df['day'] = date[2]

df.year = pd.to_numeric(df.year)
df.month = pd.to_numeric(df.month)
df.day = pd.to_numeric(df.day)

df.date = pd.to_datetime(df.date)

df.head()

In [ ]:
#Some features : 
print('Data point starts from ',df.date.min(),'\n')
print('Data point ends at ',df.date.max(),'\n')
print('Total no of countries in the data set ',len(df.country.unique()),'\n')
print('Total no of unique vaccines in the data set ',len(df.vaccines.unique()),'\n')

In [ ]:
df.info()

In [ ]:
df.country.unique()

In [ ]:
def size(m,n):
    fig = plt.gcf();
    fig.set_size_inches(m,n);

In [ ]:
#Word Art of countries : 
wordCloud = WordCloud(
    background_color='white',
    max_font_size = 50).generate(' '.join(df.country))

plt.figure(figsize=(15,7))
plt.axis('off')
plt.imshow(wordCloud)
plt.show()

In [ ]:
country_wise_total_vaccinated = {}

for country in df.country.unique() : 
    vaccinated = 0

    for i in range(len(df)) : 
        if df.country[i] == country : 
            vaccinated += df.daily_vaccinations[i]
    
    country_wise_total_vaccinated[country] = vaccinated 
#     made a seperate dict from the df 
    country_wise_total_vaccinated_df = pd.DataFrame.from_dict(country_wise_total_vaccinated,
                                                         orient='index',
                                                         columns = ['total_vaccinted_till_date'])
#     converted dict to df 
country_wise_total_vaccinated_df.sort_values(by = 'total_vaccinted_till_date', ascending = False, inplace = True)

country_wise_total_vaccinated_df


In [ ]:
fig = px.bar(country_wise_total_vaccinated_df, 
             y = 'total_vaccinted_till_date',
             x = country_wise_total_vaccinated_df.index,
             color = 'total_vaccinted_till_date',
             color_discrete_sequence= px.colors.sequential.Viridis_r
            )

fig.update_layout(
    title={
            'text' : "Vaccination till date in various countries",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Countries",
    yaxis_title="Total vaccinated",
    legend_title="Total vaccinated"
)

fig.show()

From the plot, some interesting facts are easily noticeable:

- The United States, despite all protests, has the highest number of vaccines.
- China, from where the virus started spreading, is at second.
- UK, where we have found a new variant strain of the virus, is right next.
- India, who has been supplying vaccines to the world is herself at 6th position.
Following that, we have Turkey, UAE, Germany and others.
Country wise daily vaccination :

In [ ]:
fig = px.line(df, x = 'date', y ='daily_vaccinations', color = 'country')

fig.update_layout(
    title={
            'text' : "Daily vaccination trend",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Daily Vaccinations"
)

fig.show()

### **Plot till date Function**

In [ ]:
# helper function
def plot_till_date(value1, value2, title, color1, color2) : 
    
    so_far_dict = {}
    
    for dates in df.date.unique() : 
        so_far_dict[dates], value1_count, value2_count = [], 0, 0
    
        for i in range(len(df)) : 
        
            if df.date[i] == dates : 
                value1_count += df[value1][i]
                value2_count += df[value2][i]
#         if dates not in so_far_dict.keys() : 
        
        so_far_dict[dates].append(value1_count) 
        so_far_dict[dates].append(value2_count)
    
    so_far_df = pd.DataFrame.from_dict(so_far_dict, orient = 'index', columns=[value1, value2])
    so_far_df.reset_index(inplace = True)
#     return so_far_df
    
    so_far_df.sort_values(by='index', inplace = True)
    
    plot = go.Figure(data=[go.Scatter( 
            x = so_far_df['index'], 
            y = so_far_df[value1], 
            stackgroup='one', 
            name = value1, 
            marker_color= color1), 
                               
                           go.Scatter( 
            x = so_far_df['index'], 
            y = so_far_df[value2], 
            stackgroup='one',
            name = value2, 
            marker_color= color2) 
        ]) 
    
    plot.update_layout(
    title={
            'text' : title,
            'y':0.95,
            'x':0.5
        },
        xaxis_title="Date"
    )
    
    return plot.show()

In [ ]:
#People vaccinated vs people fully vaccinated in the world : 
plot_till_date('people_fully_vaccinated', 'people_vaccinated','People vaccinated vs Fully vaccinated till date', '#c4eb28', '#35eb28')

In [ ]:
#People vaccinated vs people fully vaccinated per hundred in the world : 
plot_till_date('people_fully_vaccinated_per_hundred', 'people_vaccinated_per_hundred', 'People vaccinated vs Fully vaccinated per hundred till date', '#0938e3','#7127cc')

In [ ]:
# Function for Pie-Plot : 
def plot_pie(value, title, color) : 
    
    new_dict = {}
    
    for v in df[value].unique() : 
        value_count = 0
        
        for i in range(len(df)) : 
            if df[value][i] == v : 
                value_count += 1 
        
        new_dict[v] = value_count 
        
#     print(new_dict)


    new_df = pd.DataFrame.from_dict(new_dict, orient = 'index', columns = ['Total'])
    
    if color == 'plasma' : 
        
        fig = px.pie(new_df, values= 'Total',
                     names = new_df.index,
                     title = title,
                     color_discrete_sequence=px.colors.sequential.Plasma)
    
    elif color == 'rainbow' : 
        
        fig = px.pie(new_df, values= 'Total',
                     names = new_df.index,
                     title = title,
                     color_discrete_sequence=px.colors.sequential.Rainbow)
    
    else :
        
        fig = px.pie(new_df, values= 'Total',
                     names = new_df.index,
                     title = title)
    
    fig.update_layout(
        title={
            'y':0.95,
            'x':0.5
        },
        legend_title = value
    )
        
    return fig.show()

In [ ]:
#Most used vaccine : 
df.vaccines.unique()

In [ ]:
plot_pie('vaccines', 'Various vaccines and their uses', 'plasma')

In [ ]:
# Most frequent Sources : 
plot_pie('source_name', 'Sources', 'rainbow')

In [ ]:
#Word art of Vaccines
wordCloud = WordCloud(
    background_color='white',
    max_font_size = 50).generate(' '.join(df.vaccines))

plt.figure(figsize=(12,5))
plt.axis('off')
plt.imshow(wordCloud)
plt.show()

In [ ]:
# Daily vaccination trend per million : 
fig = px.line(df, x = 'date', y ='daily_vaccinations_per_million', color = 'country')

fig.update_layout(
    title={
            'text' : "Daily vaccination trend per million",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Daily Vaccinations per million"
)

fig.show()

In [ ]:
# Total vaccinated - India vs USA : 
india_usa = [df[df.country == 'United States'], df[df.country == 'India']]
result = pd.concat(india_usa)

fig = px.line(result, x = 'date', y ='total_vaccinations', color = 'country')

fig.update_layout(
    title={
            'text' : "Total vaccinated - India vs USA",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Total Vaccinations"
)

fig.show()

In [ ]:
# Function for Maps : 
# helper function
def plot_map(variable, title, color) : 
    
    new_dict = {}
    
    for country in df.country.unique() : 
        
        for i in range(len(df)) : 
            if df.country[i] == country : 
                new_dict[country] = df[variable][i]
    
    new_dict
    
    new_df = pd.DataFrame.from_dict(new_dict, orient='index', columns =[variable])
    
    if color == None : 
        
        place_map = px.choropleth(new_df, locations =new_df.index,
                                  locationmode = 'country names',
                                  color=variable, 
                                  hover_name= new_df.index)
    
    else : 
        
        place_map = px.choropleth(new_df, locations =new_df.index,
                                  locationmode = 'country names',
                                  color=variable, 
                                  hover_name= new_df.index,
                                  color_continuous_scale= color)
        
    place_map.update_layout(
        title_text = title,
        title_x = 0.5,
        geo=dict(showocean=True, oceancolor="#7af8ff",
                showland=True, landcolor="white",
                showframe = False))
    
    
    return place_map.show()
# Most vaccinated country : 
plot_map('total_vaccinations','Most vaccinated country', None)

In [ ]:
# Vaccines Used in Different countries : 
plot_map('vaccines','Vaccines Used in Different countries', None)

In [ ]:
# People fully vaccinated in Different countries : 
plot_map('people_fully_vaccinated','People fully vaccinated in Different countries', 'haline')

In [ ]:
# Function for Animation: 
def plot_animation(value, title, color) : 
    
    new_df = df[df.year == 2021]
    
    new_df.sort_values(by='day', inplace = True)
    
    if color == None : 
        
        place_map = px.choropleth(new_df, locations ="country",
                                    locationmode = "country names", 
                                    color=value,
                                    hover_name="country",
                                    animation_frame="day")
    
    else : 
        
        place_map = px.choropleth(new_df, locations ="country",
                                locationmode = "country names", 
                                color=value,
                                hover_name="country",
                                animation_frame="day",
                                color_continuous_scale= color)

    place_map.update_layout(
        title_text = title,
        title_x = 0.5,
        geo=dict(showocean=True, oceancolor="#7af8ff",
                showland=True, landcolor="white",
                showframe = False))
    
    return place_map.show()

In [ ]:
# Animation of daily vaccinations through January 2021 in different countries: 
plot_animation('daily_vaccinations', 'Animation of daily vaccinations through January 2021 in different countries', None)

In [ ]:
# Animation of daily vaccinations per million through January 2021 in different countries: 
plot_animation('daily_vaccinations_per_million', 'Animation of daily vaccinations per million through January 2021 in different countries', 'Plotly3')

In [ ]:
# Date vs People Vaccinated vs People Fully Vaccinated | 3D: 
fig = px.scatter_3d(df, x='date', y='people_vaccinated', z='people_fully_vaccinated',
                    color='country',
                    hover_data=['country'],
                    size = 'people_fully_vaccinated',
                    opacity=0.9, 
                    symbol = 'country')


fig.update_layout(title='Date vs People Vaccinated vs People Fully Vaccinated | 3D')

fig.update_layout(
        title={
            'y':0.95,
            'x':0.5
        }
    )

fig.show()

In [ ]:
sns.heatmap(df.corr())
plt.title('Corelation Heatmap')
plt.yticks(rotation = 0);
size(10,7)